In [ ]:
#  Copyright 2021 United Kingdom Research and Innovation
#  Copyright 2021 The University of Manchester
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:    CIL Developers

# Demo 1: Deblurring as an example of inverse problems in CIL

In this demo we show how to set up an image deblurring inverse problem in CIL and how to solve it with several choices of regularizers.

First we import all tools needed:

In [ ]:
from cil.optimisation.algorithms import FISTA
from cil.optimisation.operators import BlurringOperator
from cil.optimisation.functions import LeastSquares, L2NormSquared, L1Norm, TotalVariation, ZeroFunction
from cil.utilities import dataexample, noise
from cil.utilities.display import show2D
from cil.optimisation.utilities.callbacks import TextProgressCallback

import numpy as np

### A. Setting up the direct problem

CIL comes with a number of test images such as
- `BOAT`
- `CAMERA`
- `PEPPERS`(a colour image)
- `RESOLUTION_CHART` 
- `SIMPLE_PHANTOM_2D` 
- `RAINBOW` (a colour image)

In this demo, we load the `RESOLUTION_CHART` image:

In [ ]:
u_true = dataexample.RESOLUTION_CHART.get()
print(u_true)

We can display the image (first we define a plotting function to use for consistency throughout):

In [ ]:
def myshow(image, origin="upper", size=(8,8), fix_range=(0,1), **kwargs):
    show2D(image, origin=origin, size=size, fix_range=fix_range, **kwargs)

In [ ]:
myshow(u_true)

We see that `u_true` is an instance of the CIL `ImageData` class. More image metadata is available in the image's `geometry`:

In [ ]:
print(type(u_true))
ig = u_true.geometry
print(ig)

We consider the deblurring problem

$$ d = A u + \eta$$

where 
- $u$ is the unknown sharp image
- $A$ is a linear operator representing blurring
- $\eta$ is additive Gaussian noise
- $d$ is the blurred and noisy image.

To set up the direct problem we specify a **point spread function (PSF)** to blur the image through a convolution:

In [ ]:
# Parameters for point spread function PSF (size and std)
ks          = 5
ksigma      = 2

# Create 1D PSF and 2D as outer product, then normalise.
w           = np.exp(-np.arange(-(ks-1)/2,(ks-1)/2+1)**2/(2*ksigma**2))
w.shape     = (ks,1)
PSF         = w*np.transpose(w)
PSF         = PSF/(PSF**2).sum()
PSF         = PSF/PSF.sum()

# Display PSF as image
myshow(PSF, title="PSF", fix_range=False)

To be able to apply the blurring to our test image we specify a **BlurringOperator**:

In [ ]:
A = BlurringOperator(PSF, ig)

We apply the blurring operator to the test image and display the blurred image:

In [ ]:
u_blur = A.direct(u_true)
myshow(u_blur)

We finally add some Gaussian noise to the blurred image:

In [ ]:
u_blur_noise = noise.gaussian(u_blur, seed=10, var=0.01)
myshow(u_blur_noise)

### B. Specifying and solving the inverse problem as optimization problem

We formulate the image deblurring problem as an optimization problem to simultaneously fit to the data and enforce regularization on the solution. We use a simple least squares data-fitting term with the blurring operator and try out a few different regularizers $\Psi(u)$ in the general form optimization problem:

$$
\min_u  \| A u - d\|^{2}_{2} + \alpha\,\Psi(u)
$$

The choice of $\Psi(u)$ determines the regularization to apply. We first try without regularization, $\Psi(u) = 0$, leaving us with the simpler least-squares optimization problem
$$
\min_u  \| A u - d\|^{2}_{2}
$$

We set up a `LeastSquares` function object:

In [ ]:
F = LeastSquares(A, u_blur_noise)
print(F)

`F` is the objective or cost function that we want to minimize. As an example we evaluate it at the zero image:

In [ ]:
zero_image = ig.allocate(0.0)
F(zero_image)

And for the image of all one values - this apparently has a smaller `F` value so is closer to minimizer:

In [ ]:
F(zero_image + 1)

We set up an instance of the `FISTA` algorithm with just the `F`, specify starting point and how often to print intermediate results:

In [ ]:
alg_LS = FISTA(initial=zero_image, f=F, g=ZeroFunction(), update_objective_interval=5)

Once set up, we can then run it for some iterations and inspect the preliminary resulting image. We use a callback to print out our objective function values every `update_objective_interval` and at the end of our iterations. 

In [ ]:
alg_LS.run(10, callbacks=[TextProgressCallback()])
myshow(alg_LS.solution)

Still looking blurry and also more noisy. We try running more iterations (note how the algorithm can be resumed):

In [ ]:
alg_LS.run(10, callbacks=[TextProgressCallback()])
myshow(alg_LS.solution)

The image may be getting a bit sharper but also a lot noisier.

We need to use some regularization to handle the noise that is exploding by the inversion. We start by trying classic Tikhonov regularization
$$ \Psi(u) = \alpha \cdot \|u\|^2 = \alpha \cdot \sum_j u_j^2$$
which we specify as a CIL `L2NormSquared` function:

In [ ]:
G_L2 = 0.2 * L2NormSquared()

Another `FISTA` algorithm instance is set up, now with the regularizer passed:

In [ ]:
alg_L2 = FISTA(initial=zero_image, f=F, g=G_L2, update_objective_interval=5)

We run some iterations and display the result:

In [ ]:
alg_L2.run(20, callbacks=[TextProgressCallback()])
myshow(alg_L2.solution)

This is handling the noise better but it is still very smooth, which is the expected behaviour of Tikhonov regularization which introduces smoothing.

We try instead L1-norm regularization, which attempts to enforce sparsity of the image
$$\Psi(u)= \| u\|_1  = \sum_j |u_j| $$
which in CIL looks like

In [ ]:
G_L1 = 0.2 * L1Norm()

The `FISTA` instance is set up in the same way as before, with the new `g`:

In [ ]:
alg_L1 = FISTA(initial=zero_image, f=F, g=G_L1, update_objective_interval=5)

We run some iterations and display the result:

In [ ]:
alg_L1.run(20, callbacks=[TextProgressCallback()])
myshow(alg_L1.solution)

This produces a sharper image, with some of the noise reduced, but still very noisy.

We finally try Total Variation regularization
$$ \Psi(u) = TV(u) = \|\nabla u\|_{2,1} $$
which attempts to make the gradient image sparse, which corresponds to a piecewise constant image.

In [ ]:
G_TV = 0.02 * TotalVariation()

In [ ]:
alg_TV = FISTA(initial=zero_image, f=F, g=G_TV, update_objective_interval=1)

In [ ]:
alg_TV.run(10, callbacks=[TextProgressCallback()])
myshow(alg_TV.solution)

We see the noise is reduced a lot and the larger bars are well recovered. However the smaller bars are blending together.  One can play with the choice regularization parameter to study the trade-off.

In general, TV tends to reproduce larger features like this very well from noisy data, but lose small features.

Finally we compare all the reconstructions:

In [ ]:
myshow([u_true, u_blur_noise, alg_LS.solution,  alg_L2.solution, alg_L1.solution, alg_TV.solution],
       title=['Original', 'Blurred and noisy', 'Unregularised Least squares', 'L2',  'L1', 'TV'],
       num_cols=3)